In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from implementations import least_squares
weights, loss = least_squares(y, tX)

In [4]:
weights, loss

(array([ 8.03494319e-05, -7.20202272e-03, -6.05417273e-03, -5.47559067e-04,
        -1.93874699e-02,  4.73451619e-04, -2.60379055e-02,  3.25106300e-01,
        -3.80780232e-05, -2.72736393e+00, -2.21220140e-01,  9.50794092e-02,
         6.40351615e-02,  2.73562361e+00, -3.31801214e-04, -9.54325125e-04,
         2.74038035e+00, -5.34164961e-04,  9.73498639e-04,  3.69225052e-03,
         3.54487397e-04, -5.43344601e-04, -3.30448035e-01, -1.40800497e-03,
         8.31432879e-04,  1.02117272e-03, -1.68047417e-03, -5.83664813e-03,
        -1.11087999e-02,  2.72782386e+00]),
 0.3396868094770736)

## Generate predictions and save ouput in csv format for submission:

In [5]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [7]:
from datetime import datetime
method = 'least_squares'
time = datetime.now().strftime('%Y%m%dH%H%M%S')
OUTPUT_PATH = f'submissions/submission_{method}_{time}' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)